In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# CSV dosyasını oku
import pandas as pd
csv_file_path = '/content/drive/MyDrive/oflazer/data/yeniVeriler/datall2.csv'
data = pd.read_csv(csv_file_path)


In [12]:
data

,Comment
0,Tam kalıbınızı alabilirsiniz..biraz salaş bir ...
1,iç gösteriyor
2,dilediğiniz gibi gönül ferahlığıyla alışveriş ...
3,Hızlı bir teslimat oldu özenle katlanıp temiz ...
4,Ürün çok güzel bir beden küçük alınabilir soru...
...,...
23210,"Merhaba uygulamanız güzel, ancak çok eksik var..."
23211,Nasdaq halka arzlari da eklenebilir çok güzel ...
23212,Uygulama gerçekten çok güzel özellikle al tut ...
23213,Kripto yatırımları gelse çok iyi olur...


In [16]:
len(data)

23215

In [15]:
data['Comment'].iloc[0]

'Tam kalıbınızı alabilirsiniz..biraz salaş bir model,ben çok beğendim sipariş verdiğimin ertesi günü elime ulaştı.. hız ve kaliteniz için teşekkürler'

In [3]:
!pip install pandas transformers nltk

In [12]:
import re
import pandas as pd
from collections import Counter
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)

    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)
    return text



In [16]:

# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

# Veri ön işleme
#processed_data = [preprocess_text(data['Comment'].iloc[j]) for j in range(len(data))]

# Bigramları bulma
#bigram_measures = BigramAssocMeasures()
#bigram_finder = BigramCollocationFinder.from_words(processed_data)
#most_common_bigrams = bigram_finder.ngram_fd.most_common(20)

# Trigramları bulma
#trigram_measures = TrigramAssocMeasures()
#trigram_finder = TrigramCollocationFinder.from_words(processed_data)
#most_common_trigrams = trigram_finder.ngram_fd.most_common(20)

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=["comment", "entity_list"])

for j in range(len(data)):
    # Yorumları ön işleme
    #processed_comment = preprocess_text(comment)
    # Veri ön işleme
    comment = str(data['Comment'].iloc[j])
    processed_data = preprocess_text(comment)


    # Veri ön işleme


    # NER ile entity tespiti
    entities = ner(processed_data)
    org_entities = [entity['word'] for entity in entities if entity['entity_group'] == 'ORG']

    # En sık geçen kelimeler, bigramlar ve trigramlar
    #bigrams_in_comment = [' '.join(bigram) for bigram, freq in most_common_bigrams if all(word in processed_comment for word in bigram)]
    #trigrams_in_comment = [' '.join(trigram) for trigram, freq in most_common_trigrams if all(word in processed_comment for word in trigram)]

    # Entity listesine ekleme
    all_entities = list(set(org_entities))

    # Yeni satırı DataFrame'e ekle
    new_row = {"Comment": comment, "entity_list": all_entities}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [13]:
import re
import pandas as pd
from collections import Counter
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', ' ', text)

    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)
    return text

# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

# Veri ön işleme
processed_data = [preprocess_text(comment) for comment in data]

# Bigramları bulma
#bigram_measures = BigramAssocMeasures()
#bigram_finder = BigramCollocationFinder.from_words(processed_data)
#most_common_bigrams = bigram_finder.ngram_fd.most_common(20)

# Trigramları bulma
#trigram_measures = TrigramAssocMeasures()
#trigram_finder = TrigramCollocationFinder.from_words(processed_data)
#most_common_trigrams = trigram_finder.ngram_fd.most_common(20)

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=["comment", "entity_list"])

for comment in data:
    # Yorumları ön işleme
    #processed_comment = preprocess_text(comment)
    # Veri ön işleme
    processed_data = [preprocess_text(str(comment)) for comment in data]

    # Veri ön işleme


    # NER ile entity tespiti
    entities = ner(comment)
    org_entities = [entity['word'] for entity in entities if entity['entity_group'] == 'ORG']

    # En sık geçen kelimeler, bigramlar ve trigramlar
    #bigrams_in_comment = [' '.join(bigram) for bigram, freq in most_common_bigrams if all(word in processed_comment for word in bigram)]
    #trigrams_in_comment = [' '.join(trigram) for trigram, freq in most_common_trigrams if all(word in processed_comment for word in trigram)]

    # Entity listesine ekleme
    all_entities = list(set(org_entities))

    # Yeni satırı DataFrame'e ekle
    new_row = {"Comment": comment, "entity_list": all_entities}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


TypeError: expected string or bytes-like object

In [17]:
df

,comment,entity_list,Comment
0,NaN,[],Tam kalıbınızı alabilirsiniz..biraz salaş bir ...
1,NaN,[],iç gösteriyor
2,NaN,[],dilediğiniz gibi gönül ferahlığıyla alışveriş ...
3,NaN,[],Hızlı bir teslimat oldu özenle katlanıp temiz ...
4,NaN,[],Ürün çok güzel bir beden küçük alınabilir soru...
...,...,...,...
23210,NaN,[],"Merhaba uygulamanız güzel, ancak çok eksik var..."
23211,NaN,[nasdaq halka arzlari],Nasdaq halka arzlari da eklenebilir çok güzel ...
23212,NaN,[],Uygulama gerçekten çok güzel özellikle al tut ...
23213,NaN,[kripto],Kripto yatırımları gelse çok iyi olur...


In [ ]:
df.iloc[2000]['comment']

'Turkcell faturasız paketler her hafta pahalanıyor. Bir bakıyorum yine artmış hiç düşmüyor fiyatlar hep yükseliyor. Şebeke çekmiyor internet paketleri aldı başını gitti. Böyle giderse operatör değiştireceğim. Bu nedir ya. Bir kere de şaşırt bizi Turkcell (! )'

In [ ]:
df.iloc[2000]['entity_list']

['Turkcell']

In [18]:
df.to_csv('/content/drive/MyDrive/oflazer/data/yeniVeriler/datall_entityresult.csv', index=False, encoding='utf-8')

zimbirti için

In [19]:


# Boş bir liste oluştur
entity = []

# DataFrame üzerinde döngü
for i in range(len(df)):
  if df.iloc[i]['entity_list'] != []:
    # entity_list sütunundaki liste öğelerini al
    entity.extend(df.iloc[i]['entity_list'])
  else:
    continue

# Sonuçları göster
print("Tüm entity'ler:")
print(entity)


Tüm entity'ler:
['xl', 'm l', 'apple', 'appleın', 'lol', 'trendyol', 'macbook', 'trendyol', 'apple', 'media markt', 'apple', 'alın', 'beden', 'bedenim', 'm beden', 'fp', 'aldım s', 'fp', 'bedenininizi', 'beden', 'beden', 'oversize', 'fitness', 'beden', 'büyüyü', 'bedeninizi', 'beden', 'super ceket', 'xl', 's istemiştim', 'beden', 'express', 'gta pubg mobil', 'city bilişime', 'city bilişim', 'gta', 'city bilişim', 'city bilişime', 'valorant', 'yüz', 'koruma', 'kremi', 'aydınlatıcı', 'new well ton', 'duo', 'güneş', 'güneş', 'bebek pudrası', 'trendyol', 'parfüm', 'tt', 'tester', 'boss', 'h boss intense', 'allinonesalee', 'bebek', 'kargo', 'trendyol', 'kare', 'beden', 'beden', 'beden', 'alın', 'beden', 'bedeninizi', 'm beden', 'beden', 'duruşu', 'bedeninizi', 'xs', 's beden', 'süper ayakkabı', 'tam beden', 'iphone', 'omix', 'wifi', 'reeder', 'kvk', 'garanti sizemi', 'samsung', 'trendyol', 'dışı', 'kaçak telefon', 'niye', 'trendyol', 'bedeninizi', 'sm', 'salaş', 'kampanyasından', 'mük har e

In [20]:
set(entity)

{'bebek',
 'anime',
 'tuşu',
 'deniz bankı',
 'payfixbence',
 'stop kar',
 'gece modu',
 'android sistem güvenliğimisin',
 'kenar mahalle bankası',
 'maaş bankaniz',
 'market',
 'postamı',
 'kılıç park',
 'kartınızı',
 'oto yıkamamarket',
 'turkcell türkiyenin',
 'yazı logosu',
 'zirat mobil',
 'elektronik cüzdanlar',
 'kuveyt türk ün',
 'kredi',
 'qr işlem',
 'faizi',
 'deniz banklada',
 'departmaninizi',
 'hisse senedi işlemleri',
 'nüfus cüzdanı',
 'play projekt',
 'kredi kartımla',
 'vereyim',
 'papara şirketine',
 'doğal',
 'düşük faizli',
 'xiaomi pro plus',
 'teb bankasının',
 'karakter',
 'black',
 'a indirimleri',
 'mtv vergimi',
 'nüfus müdürlüğündeki',
 'fast',
 'hisse karşılaştırma',
 'kredi kartı puan durumunu',
 'play protect',
 'den',
 'yapı kredi bankasi',
 'bankasını',
 'vodafon pay',
 'okuyucuyu',
 'be',
 'thy',
 'emir izleme',
 'viop varant',
 'yetkilisi',
 'google paly',
 'veresiye nakti',
 'sync',
 'vodafonenet fatura kampanyası',
 'androis',
 'ramazan pidesi',
 'i

In [21]:
len(set(entity))

4257

In [22]:
# 'entitys' listesini bir .txt dosyasına kaydetme
with open('/content/drive/MyDrive/oflazer/data/yeniVeriler/datall2_entityresult_stopwords.txt', 'w') as file:
    for entitys in entity:
        file.write(f"{entitys}\n")


In [ ]:
entitys=[]
for text in entity:
    # Sayıları ve sayı içeren kelimeleri çıkarma
  text = re.sub(r'\b\d+\b', '', text)  # Sayıları çıkarır
  text = re.sub(r'\b\w*\d\w*\b', '', text)  # Sayı içeren kelimeleri çıkarır
  entitys.extend(text)

In [ ]:
len(set(entitys))

88

In [ ]:
import re
import pandas as pd
from collections import Counter
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from zımbırtı import zımbırtı_kelimeler  # zımbırtı.py dosyasını import etme

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)

    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b\d+\b', '', text)  # Sayıları çıkarır
    text = re.sub(r'\b\w*\d\w*\b', '', text)  # Sayı içeren kelimeleri çıkarır
    return text

# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

# Veri ön işleme
processed_data = [preprocess_text(comment) for comment in data]

# Bigramları bulma
bigram_measures = BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_words(processed_data)
most_common_bigrams = bigram_finder.ngram_fd.most_common(20)

# Trigramları bulma
trigram_measures = TrigramAssocMeasures()
trigram_finder = TrigramCollocationFinder.from_words(processed_data)
most_common_trigrams = trigram_finder.ngram_fd.most_common(20)

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=["comment", "entity_list"])

for comment in data:
    # Yorumları ön işleme
    processed_comment = preprocess_text(comment)

    # NER ile entity tespiti
    entities = ner(comment)
    org_entities = [entity['word'] for entity in entities if entity['entity_group'] == 'ORG']

    # En sık geçen kelimeler, bigramlar ve trigramlar
    bigrams_in_comment = [' '.join(bigram) for bigram, freq in most_common_bigrams if all(word in processed_comment for word in bigram)]
    trigrams_in_comment = [' '.join(trigram) for trigram, freq in most_common_trigrams if all(word in processed_comment for word in trigram)]

    # Entity listesine ekleme
    all_entities = list(set(org_entities + bigrams_in_comment + trigrams_in_comment))
    all_entities = [entity for entity in all_entities if entity not in zımbırtı_kelimeler]

    # Yeni satırı DataFrame'e ekle
    new_row = {"comment": comment, "entity_list": all_entities}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
